In [ ]:
import os, csv
import numpy as np
from collections import deque

def make_symmetric_2axes(seed):
    top = np.concatenate([seed, seed[:, ::-1]], axis=1)
    full = np.concatenate([top, top[::-1, :]], axis=0)
    return full

def _bfs_collect(arr, start, val):
    H, W = arr.shape
    q = deque([start]); seen = {start}
    while q:
        r, c = q.popleft()
        for dr, dc in [(-1,0),(1,0),(0,-1),(0,1)]:
            nr, nc = r+dr, c+dc
            if 0 <= nr < H and 0 <= nc < W and arr[nr, nc] == val and (nr, nc) not in seen:
                seen.add((nr, nc)); q.append((nr, nc))
    return seen

def ones_connected(arr):
    if arr.sum() == 0:
        return False
    start = tuple(np.argwhere(arr==1)[0])
    return len(_bfs_collect(arr, start, 1)) == int(arr.sum())

def has_holes(arr):
    H, W = arr.shape
    vis = np.zeros_like(arr, dtype=bool)
    q = deque()

    # enqueue border zeros
    for r in range(H):
        if arr[r, 0] == 0 and not vis[r, 0]:
            vis[r, 0] = True; q.append((r, 0))
        if arr[r, W-1] == 0 and not vis[r, W-1]:
            vis[r, W-1] = True; q.append((r, W-1))
    for c in range(W):
        if arr[0, c] == 0 and not vis[0, c]:
            vis[0, c] = True; q.append((0, c))
        if arr[H-1, c] == 0 and not vis[H-1, c]:
            vis[H-1, c] = True; q.append((H-1, c))

    # BFS over zeros
    while q:
        r, c = q.popleft()
        for dr, dc in [(-1,0),(1,0),(0,-1),(0,1)]:
            nr, nc = r+dr, c+dc
            if 0 <= nr < H and 0 <= nc < W and not vis[nr, nc] and arr[nr, nc] == 0:
                vis[nr, nc] = True; q.append((nr, nc))

    return np.any((arr == 0) & (~vis))

def corners_ok(arr, mode="any"):
    """Corner/edge constraint.
    mode="any"       -> at least one of the four corners is 1  (default)
    mode="all"       -> all four corners are 1
    mode="each_edge" -> at least one '1' on each border edge (top, bottom, left, right)
    """
    tl, tr, bl, br = int(arr[0,0]), int(arr[0,-1]), int(arr[-1,0]), int(arr[-1,-1])
    if mode == "any":
        return (tl + tr + bl + br) >= 1
    if mode == "all":
        return (tl == 1 and tr == 1 and bl == 1 and br == 1)
    if mode == "each_edge":
        top    = bool(arr[0, :].any())
        bottom = bool(arr[-1, :].any())
        left   = bool(arr[:, 0].any())
        right  = bool(arr[:, -1].any())
        return top and bottom and left and right
    raise ValueError("Unknown mode for corners_ok")

def sample_valid_cell(N=10, max_tries=20000, rng=None, p_range=(0.35, 0.65), corner_mode="any"):
    assert N % 2 == 0
    rng = np.random.default_rng() if rng is None else rng
    half = N // 2
    for _ in range(max_tries):
        p = rng.uniform(*p_range)
        seed = (rng.random((half, half)) < p).astype(np.uint8)
        full = make_symmetric_2axes(seed)
        if ones_connected(full) and not has_holes(full) and corners_ok(full, mode=corner_mode):
            return full
    return None

N = 10
TARGET = 10000
OUT_DIR = "MLMM_stress_run3"
SEED = 123
P_RANGE = (0.35, 0.65)
UNIQUE = True
CORNER_MODE = "any"     # "any" | "all" | "each_edge"

os.makedirs(OUT_DIR, exist_ok=True)
rng = np.random.default_rng(SEED)
seen = set()
meta_rows = []
count = 0
attempts = 0

last_cell = None

while count < TARGET:
    attempts += 1
    cell = sample_valid_cell(
        N=N,
        rng=rng,
        p_range=P_RANGE,
        max_tries=20000,
        corner_mode=CORNER_MODE
    )
    if cell is None:
        lo, hi = P_RANGE
        P_RANGE = (max(0.05, lo - 0.05), min(0.95, hi + 0.05))
        continue

    key = cell.tobytes()
    if UNIQUE and key in seen:
        continue
    seen.add(key)

    count += 1
    fname = f"cell_{count:05d}.csv"
    np.savetxt(os.path.join(OUT_DIR, fname), cell, fmt='%d', delimiter=',')

    ones = int(cell.sum())
    meta_rows.append([fname, ones, ones/(N*N)])

    last_cell = cell.copy()

    if count % 100 == 0:
        print(f"Saved {count}/{TARGET}")

# write metadata
with open(os.path.join(OUT_DIR, "metadata.csv"), "w", newline="") as f:
    w = csv.writer(f)
    w.writerow(["filename", "ones_count", "vol_frac"])
    w.writerows(meta_rows)

print(f"Done. Wrote {count} CSVs to '{OUT_DIR}' and a metadata.csv.")